In [1]:
import pandas as pd
import numpy as np
import os
import duckdb
from pathlib import Path
from tqdm import tqdm
from scipy.sparse import csr_matrix, hstack
from scipy.sparse.linalg import lsqr
import warnings
import plotly.graph_objects as go
import plotly.express as px
import scipy.sparse as sp
from scipy.sparse.linalg import lsqr


# --- Setup ---
warnings.filterwarnings("ignore")
DATA_FOLDER = Path("/home/akimovh/predatory_pricing_airlines/data")
PLOT_FOLDER = Path("/home/akimovh/predatory_pricing_airlines/plots")
con = duckdb.connect(str(DATA_FOLDER / "duckdb" / "airlines.duckdb"))
con.execute(f"PRAGMA threads={os.cpu_count()}")
# Create color palette
color_palette = ['rgba(0, 74, 174, 1)', 'rgba(231, 17, 94, 1)', 'rgba(255, 195, 27, 1)',  'rgba(130, 54, 199, 1)',
                 'rgba(1, 137, 67, 1)', 'rgba(252, 147, 19, 1)', 'rgba(159, 209, 255, 1)', 'rgba(119, 7, 71, 1)']

# HA revenue plot

In [2]:
costs = con.execute("""
           select t1.*,
                  make_date(t1.Year::INT, (t1.Quarter::INT - 1) * 3 + 1, 1) AS date,
            from T_F41 as t1
            where UNIQUE_CARRIER == 'HA'
                  and REGION == 'D'
            order by date
           """
          ).df()

In [3]:
# -----------------------
# Prepare data
# -----------------------
rev_plot_data = costs[['date', 'INCOME_PRE_TAX']].copy()
rev_plot_data['INCOME_PRE_TAX'] *= 1000

year_tick_vals = (
    rev_plot_data
    .groupby(rev_plot_data['date'].dt.year)['date']
    .first()
    .to_numpy()
)

unique_years = rev_plot_data['date'].dt.year.unique()

# -----------------------
# Create figure
# -----------------------
fig = go.Figure()

# Income line with markers
fig.add_trace(
    go.Scatter(
        x=rev_plot_data['date'],
        y=rev_plot_data['INCOME_PRE_TAX'],
        mode='lines+markers',
        line=dict(
            color='rgba(0, 74, 174, 1)',
            width=3
        ),
        marker=dict(
            color='rgba(0, 74, 174, 1)',
            size=10,
            symbol='circle'
        )
    )
)

# Horizontal zero line
fig.add_shape(
    type='line',
    x0=rev_plot_data['date'].min(), y0=0,
    x1=rev_plot_data['date'].max(), y1=0,
    line=dict(color='black', width=1)
)

# Southwest entry vertical line
fig.add_shape(
    type='line',
    x0=pd.to_datetime('2019-04-01'), y0=rev_plot_data['INCOME_PRE_TAX'].min(),
    x1=pd.to_datetime('2019-04-01'), y1=rev_plot_data['INCOME_PRE_TAX'].max(),
    line=dict(color='rgba(231, 17, 94, 1)', width=1.5, dash='dash')
)

fig.add_annotation(
    x=pd.to_datetime('2019-04-01'),
    y=rev_plot_data['INCOME_PRE_TAX'].max() + 10_000_000,
    text='SW entry',
    showarrow=False,
    font=dict(
        family='Arial',
        size=20,
        color='rgba(231, 17, 94, 1)'
    )
)

# COVID-19 shaded period
fig.add_shape(
    type='rect',
    x0='2020-02-01',
    x1='2021-04-01',
    y0=-200_000_000,
    y1=180_000_000,
    fillcolor='lightgray',
    opacity=0.3,
    layer='below',
    line_width=0
)

fig.add_annotation(
    x='2020-08-10',
    y=130_000_000,
    text='COVID-19',
    showarrow=False,
    align='center',
    font=dict(
        family='Arial',
        size=20,
        color='rgba(231, 17, 94, 1)'
    ),
    bgcolor='white',
    opacity=0.7,
    borderpad=4
)

# -----------------------
# Layout
# -----------------------
fig.update_layout(
    width=1280,
    height=720,
    font=dict(
        family='Arial',
        size=20
    ),
    xaxis=dict(
        title='Date (Quarter)',
        title_font=dict(size=22),
        tickmode='array',
        tickvals=year_tick_vals,
        ticktext=unique_years,
        tickfont=dict(size=18),
        gridcolor='rgba(211, 211, 211, 0.3)',
        gridwidth=1,
        showgrid=True
    ),
    yaxis=dict(
        title='Pre-tax Net Income (Millions US Dollars)',
        title_font=dict(size=22),
        tickfont=dict(size=18),
        gridcolor='rgba(211, 211, 211, 0.3)',
        gridwidth=1,
        showgrid=True,
        tickformat=','
    ),
    margin=dict(t=50, b=80, l=100, r=40),
    template='plotly_white',
    plot_bgcolor='rgba(255, 255, 255, 1)'
)

fig.show()

fig.write_image(PLOT_FOLDER / "HA_income.png",scale=3)


# Price-cost analysis

### Price

In [4]:
inner_HI = con.execute("""select t1.*,
                                 case when TkCarrier = 'HA' then 'Hawaiian Airlines'
                                      when TkCarrier = 'WN' then 'Southwest Airlines'
                                      else 'Aloha Airlines' end as TkCarrier_name
                       from DB1B_processed t1
                       where OriginState='HI' AND DestState='HI' and TkCarrier in ['HA', 'WN', 'AQ'] """).df()

In [5]:
fig = go.Figure()

# Data setup
price_plot_data = inner_HI.groupby(['date', "TkCarrier_name"], as_index=False)[['Passengers', 'sum_fares']].sum()
price_plot_data = price_plot_data.query("Passengers > 200")
price_plot_data['price'] = price_plot_data.sum_fares / price_plot_data.Passengers
price_plot_data = price_plot_data.query("date > '2016-01-01'")

fig1 = px.line(price_plot_data, x="date", y="price", color='TkCarrier_name', symbol='TkCarrier_name')

# Re-apply custom styling to traces
for i, trace in enumerate(fig1.data):
    trace.line.color = color_palette[i]
    trace.marker.color = color_palette[i]
    trace.marker.size = 10
    trace.legendgroup = trace.name
    trace.showlegend = True
    fig.add_trace(trace)

# COVID rectangle
fig.add_shape(
    type="rect",
    x0="2020-02-01",
    x1="2021-04-01",
    y0=30,
    y1=110,
    fillcolor="lightgray",
    opacity=0.3,
    layer="below",
    line_width=0,
)

# COVID annotation
fig.add_annotation(
    x='2020-09-01',
    y=100,
    text="COVID-19",
    showarrow=False,
    font=dict(
        family="Arial",
        size=20,
        color="rgba(231, 17, 94, 1)"
    ),
    align="center",
    bgcolor="white",
    opacity=0.7,
    borderpad=4,
)

# SW entry line
fig.add_shape(
    type="line",
    x0=pd.to_datetime("2019-04-01"), y0=30,
    x1=pd.to_datetime("2019-04-01"), y1=110,
    line=dict(color='rgba(231, 17, 94, 1)', width=1.5, dash="dash")
)

# SW entry annotation
fig.add_annotation(
    x=pd.to_datetime("2019-04-01"),
    y=112,
    text="SW entry",
    showarrow=False,
    font=dict(
        family="Arial",
        size=20,
        color="rgba(231, 17, 94, 1)"
    )
)

# Merger line
fig.add_shape(
    type="line",
    x0=pd.to_datetime("2023-10-01"), y0=30,
    x1=pd.to_datetime("2023-10-01"), y1=110,
    line=dict(color='rgba(231, 17, 94, 1)', width=1.5, dash="dash")
)

# Merger annotation
fig.add_annotation(
    x=pd.to_datetime("2023-10-01"),
    y=112,
    text="AA & HA merger announcment",
    showarrow=False,
    font=dict(
        family="Arial",
        size=20,
        color="rgba(231, 17, 94, 1)"
    )
)

# Layout update for presentation
fig.update_layout(
    width=1280,
    height=720,
    font=dict(
        family="Arial",
        size=20
    ),
    xaxis=dict(
        title="Date (Quarter)",
        title_font=dict(size=22),
        tickmode='array',
        tickvals=year_tick_vals,
        ticktext=unique_years,
        tickfont=dict(size=18),
        gridcolor='rgba(211, 211, 211, 0.3)',
        gridwidth=1,
        showgrid=True
    ),
    yaxis=dict(
        title="Average Fare (US dollars)",
        title_font=dict(size=22),
        tickfont=dict(size=18),
        gridcolor='rgba(211, 211, 211, 0.3)',
        gridwidth=1,
        showgrid=True
    ),
    template='plotly_white',
    plot_bgcolor='rgba(255, 255, 255, 1)',
    legend=dict(
        font=dict(size=18),
        orientation="h",
        y=-0.15,
        x=0.5,
        xanchor="center",
        bordercolor="black",
        borderwidth=1
    ),
    margin=dict(t=50, b=100, l=100, r=40)
)

fig.show()

# High-res export for LaTeX presentation
fig.write_image( PLOT_FOLDER/"HI_II_avg_fares.png", scale=3 )


### Costs

In [6]:
cost_data = con.execute("""
WITH 
-- Financials (System-wide Costs)
financials AS (
    SELECT UNIQUE_CARRIER, YEAR, QUARTER,
           (FLYING_OPS + MAINTENANCE + AIRCFT_SERVICES + TRANS_EXPENSES + PAX_SERVICE + PROMOTION_SALES) AS dist_cost,
           PAX_SERVICE AS pas_cost
    FROM T_F41
    WHERE REGION = 'D' AND UNIQUE_CARRIER IN ('WN', 'HA', 'AQ')
),

-- System-Wide Stats (The Denominator)
system_stats AS (
    SELECT UNIQUE_CARRIER, YEAR, QUARTER,
           SUM(SEATS * DISTANCE) AS system_asm,
           SUM(DEPARTURES_PERFORMED * DISTANCE) / NULLIF(SUM(DEPARTURES_PERFORMED), 0) AS system_avg_dist
    FROM T100
    WHERE DEPARTURES_PERFORMED > 0 AND CLASS IN ('F','E','C')
    GROUP BY 1, 2, 3
),

-- Target Route Stats (The Numerator)
route_stats AS (
    SELECT t1.UNIQUE_CARRIER, t1.YEAR, t1.QUARTER, t2.market_id,
           SUM(t1.PASSENGERS) / NULLIF(SUM(t1.SEATS), 0) AS load_factor,
           SUM(t1.DEPARTURES_PERFORMED * t1.DISTANCE) / NULLIF(SUM(t1.DEPARTURES_PERFORMED), 0) AS segment_dist
    FROM T100 t1
    JOIN routes t2 ON t1.ORIGIN_CITY_MARKET_ID = t2.ORIGIN_CITY_MARKET_ID 
                  AND t1.DEST_CITY_MARKET_ID = t2.DEST_CITY_MARKET_ID
    WHERE t2.market_id IN (SELECT DISTINCT market_id FROM final_panel WHERE target_routes = 1)
      AND t1.DEPARTURES_PERFORMED > 0 AND t1.CLASS IN ('F','E','C')
    GROUP BY 1, 2, 3, 4
)

-- 4. Final Calculation
SELECT 
    r.*,
    case when UNIQUE_CARRIER = 'HA' then 'Hawaiian Airlines'
         when UNIQUE_CARRIER = 'WN' then 'Southwest Airlines'
         else 'Aloha Airlines' end as TkCarrier_name, 
    make_date(r.YEAR::INT, (r.QUARTER::INT - 1) * 3 + 1, 1) AS date,
    (
        ((f.dist_cost / s.system_asm) * 1000)      -- Base CASM (cents)
        * r.segment_dist                           -- Segment Distance
        * POW(s.system_avg_dist / r.segment_dist, 0.15) -- scale adjustment
    ) / r.load_factor AS avg_cost

FROM route_stats r
JOIN financials f USING (UNIQUE_CARRIER, YEAR, QUARTER)
JOIN system_stats s USING (UNIQUE_CARRIER, YEAR, QUARTER)
where YEAR >= 2016
ORDER BY r.market_id, r.YEAR, r.QUARTER
""").df()

In [7]:
fig = go.Figure()

cost_plot_data = cost_data.groupby(['TkCarrier_name', 'date'], as_index = False).avg_cost.mean()

fig1 = px.line(cost_plot_data, x="date", y="avg_cost", color='TkCarrier_name', symbol='TkCarrier_name')

# Re-apply custom styling to traces
for i, trace in enumerate(fig1.data):
    trace.line.color = color_palette[i]
    trace.marker.color = color_palette[i]
    trace.marker.size = 10
    trace.legendgroup = trace.name
    trace.showlegend = True
    fig.add_trace(trace)

# COVID shaded region
fig.add_shape(
    type="rect",
    x0="2020-02-01",
    x1="2021-04-01",
    y0=-20,
    y1=110,
    fillcolor="lightgray",
    opacity=0.3,
    layer="below",
    line_width=0,
)

# COVID annotation
fig.add_annotation(
    x='2020-09-01',
    y=80,
    text="COVID-19",
    showarrow=False,
    font=dict(
        family="Arial",
        size=20,
        color="rgba(231, 17, 94, 1)"
    ),
    align="center",
    bgcolor="white",
    opacity=0.7,
    borderpad=4,
)

# Southwest entry vertical line
fig.add_shape(
    type="line",
    x0=pd.to_datetime("2019-04-01"), y0=-1000,
    x1=pd.to_datetime("2019-04-01"), y1=100,
    line=dict(color='rgba(231, 17, 94, 1)', width=1.5, dash="dash")
)

# Southwest entry annotation
fig.add_annotation(
    x=pd.to_datetime("2019-04-01"),
    y=102,
    text="SW entry",
    showarrow=False,
    font=dict(
        family="Arial",
        size=20,
        color='rgba(231, 17, 94, 1)'
    )
)

# Layout
fig.update_layout(
    width=1280,
    height=720,
    font=dict(
        family="Arial",
        size=20
    ),
    xaxis=dict(
        title="Date (Quarter)",
        title_font=dict(size=22),
        tickmode='array',
        tickvals=year_tick_vals,
        ticktext=unique_years,
        tickfont=dict(size=18),
        gridcolor='rgba(211, 211, 211, 0.3)',
        gridwidth=1,
        showgrid=True
    ),
    yaxis=dict(
        title="Average Cost per Passenger (US dollars)",
        title_font=dict(size=22),
        tickfont=dict(size=18),
        gridcolor='rgba(211, 211, 211, 0.3)',
        gridwidth=1,
        showgrid=True,
        range=[0, 105]
    ),
    template='plotly_white',
    plot_bgcolor='rgba(255, 255, 255, 1)',
    legend=dict(
        font=dict(size=18),
        orientation="h",
        y=-0.15,
        x=0.5,
        xanchor="center",
        bordercolor="black",
        borderwidth=1
    ),
    margin=dict(t=50, b=100, l=100, r=40)
)

fig.show()

fig.write_image( PLOT_FOLDER/"HI_II_avg_cost.png", scale=3 )

In [8]:
fig = go.Figure()

markup_plot_data = pd.merge(price_plot_data, cost_plot_data, on = ['date', 'TkCarrier_name'])
markup_plot_data["avg_markup"] = markup_plot_data.price - markup_plot_data.avg_cost 
fig1 = px.line(markup_plot_data, x="date", y="avg_markup", color='TkCarrier_name', symbol='TkCarrier_name')

# Re-apply custom styling to traces
for i, trace in enumerate(fig1.data):
    trace.line.color = color_palette[i]
    trace.marker.color = color_palette[i]
    trace.marker.size = 10
    trace.legendgroup = trace.name
    trace.showlegend = True
    fig.add_trace(trace)

# COVID rectangle
fig.add_shape(
    type="rect",
    x0="2020-02-01",
    x1="2021-04-01",
    y0=-55,
    y1=110,
    fillcolor="lightgray",
    opacity=0.3,
    layer="below",
    line_width=0,
)

# COVID annotation
fig.add_annotation(
    x='2020-09-01',
    y=80,
    text="COVID-19",
    showarrow=False,
    font=dict(
        family="Arial",
        size=20,
        color="rgba(231, 17, 94, 1)"
    ),
    align="center",
    bgcolor="white",
    opacity=0.7,
    borderpad=4,
)

# Horizontal line at y = 0
fig.add_shape(
    type="line",
    x0=markup_plot_data['date'].min(),
    x1=markup_plot_data['date'].max(),
    y0=0,
    y1=0,
    line=dict(
        color="black",
        width=2,
        dash="solid"
    )
)

# Southwest entry line
fig.add_shape(
    type="line",
    x0=pd.to_datetime("2019-04-01"),
    x1=pd.to_datetime("2019-04-01"),
    y0=-1000,
    y1=100,
    line=dict(color='rgba(231, 17, 94, 1)', width=1.5, dash="dash")
)

# Southwest entry annotation
fig.add_annotation(
    x=pd.to_datetime("2019-04-01"),
    y=102,
    text="SW entry",
    showarrow=False,
    font=dict(
        family="Arial",
        size=20,
        color='rgba(231, 17, 94, 1)'
    )
)

# Layout adjustments
fig.update_layout(
    width=1280,
    height=720,
    font=dict(
        family="Arial",
        size=20
    ),
    xaxis=dict(
        title="Date (Quarter)",
        title_font=dict(size=22),
        tickmode='array',
        tickvals=year_tick_vals,
        ticktext=unique_years,
        tickfont=dict(size=18),
        gridcolor='rgba(211, 211, 211, 0.3)',
        gridwidth=1,
        showgrid=True
    ),
    yaxis=dict(
        title="Average Markup (US dollars)",
        title_font=dict(size=22),
        tickfont=dict(size=18),
        gridcolor='rgba(211, 211, 211, 0.3)',
        gridwidth=1,
        showgrid=True,
        range=[-50, 105]
    ),
    template='plotly_white',
    plot_bgcolor='rgba(255, 255, 255, 1)',
    legend=dict(
        font=dict(size=18),
        orientation="h",
        y=-0.15,
        x=0.5,
        xanchor="center",
        bordercolor="black",
        borderwidth=1
    ),
    margin=dict(t=50, b=100, l=100, r=40)
)

fig.show()

# Export high-resolution image
fig.write_image( PLOT_FOLDER/"HI_II_avg_markup.png", scale=3 )

# Event study

In [78]:
# Load Data
df = con.execute("select * from final_panel").df()

y_full = df['mean_log_fare'].values

df['date_cat'] = df['date'].astype('category')
df['market_cat'] = df['market_id'].astype('category')

n_rows = len(df)

# Constant
X_const = sp.csc_matrix(np.ones((n_rows, 1)))

# Time Fixed Effects (One-Hot Encoding)
X_time = sp.coo_matrix(
    (np.ones(n_rows), (np.arange(n_rows), df['date_cat'].cat.codes)),
    shape=(n_rows, len(df['date_cat'].cat.categories))
)

# Market Fixed Effects
X_market = sp.coo_matrix(
    (np.ones(n_rows), (np.arange(n_rows), df['market_cat'].cat.codes)),
    shape=(n_rows, len(df['market_cat'].cat.categories))
)

# COVID State Fixed Effects
# Logic: (Origin==State | Dest==State) & covid==1
# Build a matrix for Origin and Dest, sum them, clip to 1 (OR logic), then apply covid mask.

# Get all unique states from both columns to ensure alignment
all_states = np.unique(np.concatenate([df.OriginState.unique(), df.DestState.unique()]))
state_cat = pd.CategoricalDtype(categories=all_states, ordered=True)

orig_codes = df['OriginState'].astype(state_cat).cat.codes
dest_codes = df['DestState'].astype(state_cat).cat.codes

# Matrix of Origins
X_orig = sp.coo_matrix(
    (np.ones(n_rows), (np.arange(n_rows), orig_codes)),
    shape=(n_rows, len(all_states))
)

# Matrix of Dests
X_dest = sp.coo_matrix(
    (np.ones(n_rows), (np.arange(n_rows), dest_codes)),
    shape=(n_rows, len(all_states))
)

# Combine: (Origin OR Dest)
X_state = X_orig + X_dest
X_state.data = np.where(X_state.data > 0, 1, 0) # Boolean OR logic (handles same-state case)

# Apply COVID mask: Zero out rows where covid != 1
# Use multiply by diagonal matrix for speed
covid_diag = sp.diags(df['covid'].values)
X_covid = covid_diag.dot(X_state)

# Combine all parts horizontally
# Drop first column of Time and Market to avoid multicollinearity (Dropping reference)
# Slicing CSC is efficient for columns
X_time_csc = X_time.tocsc()[:, 1:] 
X_market_csc = X_market.tocsc()[:, 1:]

# Final X Matrix
X_full = sp.hstack([X_const, X_time_csc, X_market_csc, X_covid], format='csr')

train_mask_full = (
    ((df.rel_period < -3) | (df.rel_period == -100)) & 
    ((df.hi_routes == 0) | (df.rel_period < -6))
).values

meta_df = df[['rel_period', 'target_routes', 'monopoly_market']].reset_index(drop=True)

In [ ]:
def estimate_coefficients_optimized(idx_mask, y=y_full, X=X_full, 
                                    train_mask=train_mask_full, meta=meta_df):
    # Slice EVERYTHING using the same indices
    y_boot = y[idx_mask]
    X_boot = X[idx_mask]
    train_boot = train_mask[idx_mask]
    meta_boot = meta.iloc[idx_mask].copy() 

    # Filter for Training
    X_train = X_boot[train_boot]
    y_train = y_boot[train_boot]

    # Solve (Sparse LSQR)
    beta = lsqr(X_train, y_train)[0]

    # Predict
    y_hat = X_boot.dot(beta)
    
    # Calculate Residuals
    meta_boot['dif'] = y_boot - y_hat

    # Aggregate
    res_df = meta_boot.query("rel_period > -17 and rel_period < 17")
    
    base_coefs = res_df.query('target_routes == 0').groupby('rel_period')['dif'].mean()
    monopoly_coefs = res_df.query('target_routes == 0 and monopoly_market == 1').groupby('rel_period')['dif'].mean()
    targeted_coefs = res_df.query('target_routes == 1').groupby('rel_period')['dif'].mean()
    
    return base_coefs, monopoly_coefs, targeted_coefs

In [80]:
full_sample_mask = np.arange(len(df))
base_coefs_main, monopoly_coefs_main, targeted_coefs_main = estimate_coefficients_optimized(full_sample_mask)

In [86]:
periods = np.arange(-16, 17) 
n_periods = len(periods)
n_bootstraps = 10  # Start small as requested

# Initialize storage arrays (n_bootstraps x n_periods)
base_bootstrap = np.full((n_bootstraps, n_periods), np.nan)
monopoly_bootstrap = np.full((n_bootstraps, n_periods), np.nan)
targeted_bootstrap = np.full((n_bootstraps, n_periods), np.nan)

market_indices_map = df.groupby('market_id').indices

targeted_markets = df[df.target_routes == 1]['market_id'].unique()
non_targeted_markets = df[df.target_routes == 0]['market_id'].unique()

In [ ]:
# SETUP & PRE-COMPUTATION ---
periods = np.arange(-16, 17) 
n_periods = len(periods)
n_bootstraps = 1000  # Scaling up!

# Initialize storage
base_bootstrap = np.full((n_bootstraps, n_periods), np.nan)
monopoly_bootstrap = np.full((n_bootstraps, n_periods), np.nan)
targeted_bootstrap = np.full((n_bootstraps, n_periods), np.nan)

# Map & Lists (Run Once)
market_indices_map = df.groupby('market_id').indices
targeted_markets = df[df.target_routes == 1]['market_id'].unique()
non_targeted_markets = df[df.target_routes == 0]['market_id'].unique()

# --- SET SEED FOR REPRODUCIBILITY ---
# 42 is the answer to the ultimate question of life, the universe, and everything
np.random.seed(42) 

print(f"Starting {n_bootstraps} iterations...")

# --- 2. BOOTSTRAP LOOP ---
from tqdm import tqdm

for b in tqdm(range(n_bootstraps)):
    try:
        # A. Sample Markets (Cluster Resampling)
        samp_target_mkts = np.random.choice(targeted_markets, size=len(targeted_markets), replace=True)
        samp_nontarget_mkts = np.random.choice(non_targeted_markets, size=len(non_targeted_markets), replace=True)
        
        # B. Build Index Mask
        target_indices = [market_indices_map[m] for m in samp_target_mkts]
        nontarget_indices = [market_indices_map[m] for m in samp_nontarget_mkts]
        idx_mask = np.concatenate(target_indices + nontarget_indices)
        
        # C. Estimate
        base, mono, targ = estimate_coefficients_optimized(idx_mask)
        
        # D. Store
        base_bootstrap[b, :] = base.reindex(periods).values
        monopoly_bootstrap[b, :] = mono.reindex(periods).values
        targeted_bootstrap[b, :] = targ.reindex(periods).values

    except Exception as e:
        print(f"Error in bootstrap {b}: {e}")
        continue

print("Bootstrap complete!")

Starting 1000 iterations...


 13%|█▎        | 130/1000 [03:17<22:14,  1.53s/it]

In [88]:
# Calculate mean and confidence intervals (95%)
# We use 'nan*' functions to ignore missing periods without crashing
base_mean = np.nanmean(base_bootstrap, axis=0)
base_ci_lower = np.nanpercentile(base_bootstrap, 2.5, axis=0)
base_ci_upper = np.nanpercentile(base_bootstrap, 97.5, axis=0)

monopoly_mean = np.nanmean(monopoly_bootstrap, axis=0)
monopoly_ci_lower = np.nanpercentile(monopoly_bootstrap, 2.5, axis=0)
monopoly_ci_upper = np.nanpercentile(monopoly_bootstrap, 97.5, axis=0)

targeted_mean = np.nanmean(targeted_bootstrap, axis=0)
targeted_ci_lower = np.nanpercentile(targeted_bootstrap, 2.5, axis=0)
targeted_ci_upper = np.nanpercentile(targeted_bootstrap, 97.5, axis=0)

# Replace any remaining NaNs with zeros ONLY for plotting
# (e.g., if a period was missing in 100% of bootstraps)
base_mean = np.nan_to_num(base_mean)
base_ci_lower = np.nan_to_num(base_ci_lower)
base_ci_upper = np.nan_to_num(base_ci_upper)

monopoly_mean = np.nan_to_num(monopoly_mean)
monopoly_ci_lower = np.nan_to_num(monopoly_ci_lower)
monopoly_ci_upper = np.nan_to_num(monopoly_ci_upper)

targeted_mean = np.nan_to_num(targeted_mean)
targeted_ci_lower = np.nan_to_num(targeted_ci_lower)
targeted_ci_upper = np.nan_to_num(targeted_ci_upper)

In [ ]:
periods = list(range(-14,15))

fig = go.Figure()

# Base coefficient line and CI band
fig.add_trace(go.Scatter(
    x=periods,
    y=np.exp(base_coefs_main[periods]),
    mode='lines+markers',
    line=dict(color=color_palette[0], width=2),
    marker=dict(size=8, color=color_palette[0]),
    name='Not Intra-Hawaiian routes'
))

fig.add_trace(go.Scatter(
    x=periods + periods[::-1],
    y=np.exp(base_ci_upper[2:-2]).tolist() + np.exp(base_ci_lower[2:-2]).tolist()[::-1],
    fill='toself',
    fillcolor=color_palette[0],
    opacity=0.3,
    line=dict(color=color_palette[0]),
    hoverinfo="skip",
    showlegend=False
))

# Targeted coefficient line and CI band
fig.add_trace(go.Scatter(
    x=periods,
    y=np.exp(targeted_coefs_main[periods]),
    mode='lines+markers',
    line=dict(color=color_palette[1], width=2),
    marker=dict(size=8, color=color_palette[1]),
    name='Intra-Hawaiian routes'
))

fig.add_trace(go.Scatter(
    x=periods + periods[::-1],
    y=np.exp(targeted_ci_upper[2:-2]).tolist() + np.exp(targeted_ci_lower[2:-2]).tolist()[::-1],
    fill='toself',
    fillcolor=color_palette[1],
    opacity=0.3,
    line=dict(color=color_palette[1]),
    hoverinfo="skip",
    showlegend=False
))

# Monopoly coefficient line and CI band
fig.add_trace(go.Scatter(
    x=periods,
    y=np.exp(monopoly_coefs_main[periods]),
    mode='lines+markers',
    line=dict(color=color_palette[4], width=2),
    marker=dict(size=8, color=color_palette[4]),
    name='Markets that were monopoly before SW entry, excluding intra-Hawaiian'
))

fig.add_trace(go.Scatter(
    x=periods + periods[::-1],
    y=np.exp(monopoly_ci_upper[2:-2]).tolist() + np.exp(monopoly_ci_lower[2:-2]).tolist()[::-1],
    fill='toself',
    fillcolor=color_palette[4],
    opacity=0.3,
    line=dict(color=color_palette[4]),
    hoverinfo="skip",
    showlegend=False
))

# Reference lines
fig.add_shape(
    type="line",
    x0=min(periods), y0=1, x1=max(periods), y1=1,
    line=dict(color="black", width=2, dash="solid")
)

y_min = min(np.min(np.exp(base_ci_lower)), np.min(np.exp(targeted_ci_lower)), np.min(np.exp(monopoly_ci_lower)))
y_max = max(np.max(np.exp(base_ci_upper)), np.max(np.exp(targeted_ci_upper)), np.max(np.exp(monopoly_ci_upper)))

fig.add_shape(
    type="line",
    x0=-0.5, y0=y_min, x1=-0.5, y1=y_max,
    line=dict(color='rgba(231, 17, 94, 1)', width=1.5, dash="dash")
)

# Southwest entry annotation
fig.add_annotation(
    x=-0.5,
    y=y_max + 0.05,
    text="SW entry",
    showarrow=False,
    font=dict(
        family="Arial",
        size=20,
        color='rgba(231, 17, 94, 1)'
    )
)

# Layout settings
fig.update_layout(
    width=1280,
    height=720,
    font=dict(
        family="Arial",
        size=20
    ),
    xaxis=dict(
        title='Time Relative to Southwest Entry (Quarters)',
        title_font=dict(size=22),
        tickmode='array',
        tickvals=periods,
        tickfont=dict(size=18),
        gridcolor='rgba(211, 211, 211, 0.3)',
        gridwidth=1,
        showgrid=True
    ),
    yaxis=dict(
        title='Change in Average Market Price (Percent)',
        title_font=dict(size=22),
        tickfont=dict(size=18),
        gridcolor='rgba(211, 211, 211, 0.3)',
        gridwidth=1,
        showgrid=True,
        tickformat='.1%'
    ),
    legend=dict(
        font=dict(size=18),
        orientation="h",
        y=-0.15,
        x=0.5,
        xanchor="center",
        bordercolor="black",
        borderwidth=1
    ),
    template='plotly_white',
    plot_bgcolor='rgba(255, 255, 255, 1)',
    margin=dict(t=50, b=100, l=100, r=40)
)

fig.show()

fig.write_image( PLOT_FOLDER/"main_plot.png", scale=3 )


# Robustness checks 

### Placebo test